In [3]:
import requests
import re
from bs4 import BeautifulSoup


URL = "https://fijivillage.com/news"
page = requests.get(URL)

wholepage = BeautifulSoup(page.content, "html.parser")


In [48]:

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words



LANGUAGE = "english"
SENTENCES_COUNT = 3

def summarizetext(text):
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))

    
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    paragraph = ""

    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        paragraph += str(sentence)

    return paragraph


In [50]:
def extractinfo(divdata):
    try:
        soup = BeautifulSoup(str(divdata))
        
        # Extract the article URL and title
        article_link = soup.find('a', href=True)
        article_url = article_link['href'] if article_link else None
        title_tag = soup.find('h6')
        article_title = title_tag.text.strip() if title_tag else None
        
        # Extract the image URL
        img_tag = soup.find('img')
        image_url = img_tag.get('src') if img_tag else None

        # get article from url
        page = BeautifulSoup(requests.get(article_url).content, "html.parser")
        article = page.find("div", class_="news_reader")

        date_span = page.find('span', class_="greytime2")
        date = date_span.get_text().split(" ")[1]
        
        summary = summarizetext(article.get_text())

        
        return {
            'id':article_url,
            'title': article_title,
            'summary': summary,
            'article_url': article_url,
            'publish_time': date,
            'image_url': image_url,
        }
    except Exception as e:
        print(f"Error parsing HTML snippet: {e}")
        return None

In [52]:
newsdivs = wholepage.find_all("div", class_="col-md-4 pt-2")[:1]
news = []

for div in newsdivs:
    news.append(extractinfo(divdata=div))

print(news[0]["summary"])

FICAC has referred the matter against its Commissioner Barbara Malimali to the Office of the Director of Public Prosecutions for further assessment while FICAC has closed five cases regarding a FijiFirst declaration, Voreqe Bainimarama, Aiyaz Sayed-Khaiyum and President Ratu Wiliame Katonivere.FICAC reiterates that the Commission is operating as usual and all processes and procedures remain unchanged.FICAC says it continues to handle every complaint, regardless of who the subject person is, with the scrutiny it deserves.
